# Installation

For installation instructions please refer to `Simply laced diagrams` notebook.

In [1]:
using Pkg

In [2]:
Pkg.activate(joinpath(@__DIR__, ".."))

  Activating project at `~/Mathematics/Research/Property (T)/Chevalley/2306.12358`


In [3]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × AMD Ryzen 7 7840U w/ Radeon  780M Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, znver3)
Threads: 8 default, 0 interactive, 4 GC (on 16 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8


In [4]:
Pkg.status()

Status `~/Mathematics/Research/Property (T)/Chevalley/2306.12358/Project.toml`
  [c7e460c6] ArgParse v1.2.0
  [1e616198] COSMO v0.8.9
  [5d8bd718] Groups v0.8.0
  [7073ff75] IJulia v1.25.0
  [4076af6c] JuMP v1.22.2
  [03b72c93] PropertyT v0.6.0 `https://github.com/kalmarek/PropertyT.jl#master`
  [c946c3f1] SCS v2.0.0
⌃ [3f2553a9] SCS_MKL_jll v3.2.4+1 ⚲
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [5]:
using Groups
import Groups.MatrixGroups

In [6]:
using PropertyT
using PropertyT.IntervalArithmetic

> In this notebook only rudimentary commentary is included. For the extended one please consult `Simply laced diagrams` notebook.

# Sum of squares proof of property (T) for $\operatorname{Sp}_{4}(\mathbb{Z})$

We wish to prove
> **Theorem 3.12** Let $G= \operatorname{Sp}_{4}(\mathbb{Z})$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{B}_{\texttt{2}} = \texttt{C}_{\texttt{2}}$ endowed with the set of Steinberg generators $S$. The pair $(G, S)$ has property (T) with a witness of type $(\lambda, R) = (0.87890, 2)$.

We will show this by exhibiting $\xi_i\in \mathbb{R}G$, supported inside $\operatorname{Ball}(S, 2)$ such that

$$
\Delta^2 - \lambda \Delta - \sum_i \xi_i^* \xi_i = r,
$$

with $\|r\|_1$ much smaller (a few orders of magnitue) than $\lambda$.

In [7]:
N = 2
G = MatrixGroups.SymplecticGroup{2*N}(Int8)

group of 4×4 symplectic matrices

In [8]:
gens(G, 1)

A₁.₂ ∈ Sp{4,Int8}
 1  1   0  0
 0  1   0  0
 0  0   1  0
 0  0  -1  1

In [9]:
gens(G, 3)

B₃.₁ ∈ Sp{4,Int8}
 1  0  0  0
 0  1  0  0
 1  0  1  0
 0  0  0  1

In [10]:
gens(G, 5)

B₃.₂ ∈ Sp{4,Int8}
 1  0  0  0
 0  1  0  0
 0  1  1  0
 1  0  0  1

In [11]:
HALFRADIUS = 2
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

  0.031126 seconds (462.12 k allocations: 18.141 MiB, 40.13% gc time)
  0.004681 seconds (72.70 k allocations: 2.806 MiB)
  0.741827 seconds (2.00 M allocations: 121.676 MiB, 4.65% gc time, 95.06% compilation time)


[ Info: generating wl-metric ball of radius 4
[ Info: sizes = [17, 209, 2073, 18313]
[ Info: computing the *-algebra structure for G


In [12]:
S

16-element Vector{FPGroupElement{Groups.MatrixGroups.SymplecticGroup{4, Int8, DataType, Groups.MatrixGroups.ElementarySymplectic{4, Int8}}, KnuthBendix.Words.Word{UInt8}}}:
 A₁.₂
 A₂.₁
 B₃.₁
 B₄.₂
 B₃.₂
 B₁.₃
 B₂.₄
 B₂.₃
 A₁.₂^-1
 A₂.₁^-1
 B₃.₁^-1
 B₄.₂^-1
 B₃.₂^-1
 B₁.₃^-1
 B₂.₄^-1
 B₂.₃^-1

In [13]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

16·(id) -1·A₁.₂ -1·A₂.₁ -1·B₃.₁ -1·B₄.₂ -1·B₃.₂ -1·B₁.₃ -1·B₂.₄ -1·B₂.₃ -1·A₁.₂^-1 -1·A₂.₁^-1 -1·B₃.₁^-1 -1·B₄.₂^-1 -1·B₃.₂^-1 -1·B₁.₃^-1 -1·B₂.₄^-1 -1·B₂.₃^-1

## Optimization problem

In [14]:
@time model = PropertyT.sos_problem_primal(
    Δ^2,
    Δ,
    augmented = true,
)

  1.690365 seconds (5.49 M allocations: 380.104 MiB, 6.31% gc time, 90.15% compilation time: <1% of which was recompilation)


A JuMP Model
Maximization problem with:
Variables: 21946
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 18313 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: P, psd, λ

However, since we know the action of the Weyl group, we do not want to solve this formulation.

### Symmetry reduction

In [15]:
import PropertyT.SA as StarAlgebras
import PropertyT.SW as SymbolicWedderburn
using PropertyT.PG # PermutationGroups

In [16]:
wd = let RG = RG, N = N
    G = StarAlgebras.object(RG)
    P = PermGroup(perm"(1,2)", Perm(circshift(1:N, -1)))
    Weyl = Groups.Constructions.WreathProduct(PermGroup(perm"(1,2)"), P)
    act = PropertyT.action_by_conjugation(G, Weyl)

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Weyl,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
    )
end
@info wd

  8.087853 seconds (15.74 M allocations: 1.035 GiB, 2.84% gc time, 246.91% compilation time: <1% of which was recompilation)


┌ Info: Wedderburn Decomposition into 4830 orbits and 4 simple summands of sizes
└ [71, 58, 44, 36]


In [17]:
@time model, varP = PropertyT.sos_problem_primal(Δ^2, Δ, wd; augmented = true);
model

  1.929635 seconds (5.31 M allocations: 600.561 MiB, 8.92% gc time, 242.29% compilation time)


A JuMP Model
Maximization problem with:
Variables: 5924
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 4829 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ

In [18]:
include(joinpath(@__DIR__, "..", "src", "optimizers.jl"));
with_optimizer = scs_optimizer(;
    eps = 1e-9,
    max_iters = 20_000,
    accel = 50,
    alpha = 1.95,
);

In [19]:
warm = nothing

In [20]:
status, warm = PropertyT.solve(model, with_optimizer, warm)
@info "Optimization has finished with" status

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 5924, constraints m: 10752
cones: 	  z: primal zero / dual free vars: 4829
	  s: psd vars: 5923, ssize: 4
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.95, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 20000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 50, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-mkl-pardiso
	  nnz(A): 46608, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.21e+01  1.00e+00  1.06e+02 -1.13e+02  1.00e-01  4.44e-02 
   250| 7.51e-03  2.64e-04  1.41e-01 

┌ Info: Optimization has finished with
└   status = OPTIMAL::TerminationStatusCode = 1


### Reconstructing and certifying the solution
Since we symmetrized our problem, we don't have direct access to `P` or `Q` as defined above. So first we need to reconstruct `Q` from the action of `Weyl` on $\operatorname{Ball}(S, R)$:

In [21]:
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

 13.479702 seconds (24.32 M allocations: 1.457 GiB, 3.36% gc time, 101.22% compilation time)


209×209 Matrix{Float64}:
 0.0   0.0          0.0          0.0         …   0.0           0.0
 0.0   2.32376      0.240808     0.149926       -0.00871263    0.014647
 0.0   0.240808     2.32376     -0.0270742      -0.00871263    0.014647
 0.0   0.149926    -0.0270742    2.92568        -0.0101342    -0.00672851
 0.0  -0.0270742    0.149926     0.10302        -0.0101342    -0.00672851
 0.0   0.0689792    0.0689792   -0.0270742   …  -0.00898902    0.00720565
 0.0  -0.0270742    0.149926    -0.0792395       0.0298759     0.0692953
 0.0   0.149926    -0.0270742    0.152797        0.0298759     0.0692953
 0.0   0.0689792    0.0689792    0.149926       -0.00517579    0.00692666
 0.0   0.184116    -0.0699972    0.149926       -0.00871263    0.014647
 0.0  -0.0699972    0.184116    -0.0270742   …  -0.00871263    0.014647
 0.0   0.149926    -0.0270742    0.231367       -0.0101342    -0.00672855
 0.0  -0.0270742    0.149926     0.152797       -0.0101342    -0.00672855
 ⋮                            

In [22]:
@info "certifying the solution"
certified, λ = PropertyT.certify_solution(
    Δ^2,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)

if certified && λ > 0
    @info "Certified result: Sp_{$(2N)}(ℤ) has property (T)" PropertyT.IntervalArithmetic.inf(λ)
else
    @info "Could NOT certify the Property(T) for Sp_{$(2N)}(ℤ)" certified λ
end

[ Info: certifying the solution
┌ Info: Checking in Float64 arithmetic with
└   λ = 0.8791635045476334


  0.000639 seconds (6 allocations: 484.656 KiB)


┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ≈ -2.2393e-12
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ≈ 3.44416e-7
└  λ ≈ 0.879162126883405
┌ Info: Checking in Interval{Float64} arithmetic with
└   λ_int = [0.879163, 0.879164]


  0.303985 seconds (225.64 k allocations: 22.970 MiB, 8.95% gc time, 98.24% compilation time)


┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ∈ [-7.03357e-10, 7.12468e-10]
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ∈ [3.43701e-07, 3.45118e-07]
└  λ ∈ [0.879162, 0.879163]
┌ Info: Certified result: Sp_{4}(ℤ) has property (T)
└   PropertyT.IntervalArithmetic.inf(λ) = 0.8791621240787585


# Levels in $\operatorname{Sp}_{6}(\mathbb{Z})$

We wish here to prove
> **Theorem 3.15**: Let $G= \operatorname{Sp}_{6}(\mathbb{Z})$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{C}_\texttt{3}$ endowed with the set of Steinberg generators $S$. Let $V$ denote the ambient vector space of the root system. Then for $\lambda=2.41212$ we have
>
>$$ \operatorname{Lev}_2^3 + \operatorname{Lev}_3^3 -\lambda \Delta_V = \sum \xi_i^* \xi_i$$
>
>for some $\xi_i\in \mathbb{R}G$ supported inside $\operatorname{Ball}(S, 2)$.

In [23]:
N = 3
G = MatrixGroups.SymplecticGroup{2*N}(Int8)

group of 6×6 symplectic matrices

In [24]:
HALFRADIUS = 2
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

[ Info: generating wl-metric ball of radius 4


  1.359282 seconds (12.08 M allocations: 598.559 MiB, 18.87% gc time)


[ Info: sizes = [37, 961, 21183, 424899]
[ Info: computing the *-algebra structure for G


  0.180161 seconds (1.69 M allocations: 94.108 MiB)
  2.094619 seconds (14.78 M allocations: 759.727 MiB, 12.25% gc time, 26.46% compilation time)


In [25]:
S

36-element Vector{FPGroupElement{Groups.MatrixGroups.SymplecticGroup{6, Int8, DataType, Groups.MatrixGroups.ElementarySymplectic{6, Int8}}, KnuthBendix.Words.Word{UInt8}}}:
 A₁.₂
 A₁.₃
 A₂.₁
 A₂.₃
 A₃.₁
 A₃.₂
 B₄.₁
 B₅.₂
 B₆.₃
 B₄.₂
 B₄.₃
 B₅.₃
 B₁.₄
 ⋮
 B₄.₁^-1
 B₅.₂^-1
 B₆.₃^-1
 B₄.₂^-1
 B₄.₃^-1
 B₅.₃^-1
 B₁.₄^-1
 B₂.₅^-1
 B₃.₆^-1
 B₂.₄^-1
 B₃.₄^-1
 B₃.₅^-1

In [26]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

36·(id) -1·A₁.₂ -1·A₁.₃ -1·A₂.₁ -1·A₂.₃ -1·A₃.₁ -1·A₃.₂ -1·B₄.₁ -1·B₅.₂ -1·B₆.₃ -1·B₄.₂ -1·B₄.₃ -1·B₅.₃ -1·B₁.₄ -1·B₂.₅ -1·B₃.₆ -1·B₂.₄ -1·B₃.₄ -1·B₃.₅ -1·A₁.₂^-1 -1·A₁.₃^-1 -1·A₂.₁^-1 -1·A₂.₃^-1 -1·A₃.₁^-1 -1·A₃.₂^-1 -1·B₄.₁^-1 -1·B₅.₂^-1 -1·B₆.₃^-1 -1·B₄.₂^-1 -1·B₄.₃^-1 -1·B₅.₃^-1 -1·B₁.₄^-1 -1·B₂.₅^-1 -1·B₃.₆^-1 -1·B₂.₄^-1 -1·B₃.₄^-1 -1·B₃.₅^-1

## Grading

Let us define a $\texttt{C}_\texttt{2}$ grading for $(G, S)$. That is, to every generator $s \in S$ we need to assign a root in $\texttt{C}_\texttt{2}$ in a compatible fashion. For a precise description of this grading see **Example 3.2 (2)**.  

In [27]:
g = gens(G, 1)

A₁.₂ ∈ Sp{6,Int8}
 1  1  0   0  0  0
 0  1  0   0  0  0
 0  0  1   0  0  0
 0  0  0   1  0  0
 0  0  0  -1  1  0
 0  0  0   0  0  1

In [28]:
PropertyT.grading(g)

Root in ℝ^3 of length √2
[1, -1, 0]

In [29]:
g = gens(G, 7)

B₄.₁ ∈ Sp{6,Int8}
 1  0  0  0  0  0
 0  1  0  0  0  0
 0  0  1  0  0  0
 1  0  0  1  0  0
 0  0  0  0  1  0
 0  0  0  0  0  1

In [30]:
PropertyT.grading(g)

Root in ℝ^3 of length 2
[-2, 0, 0]

In [31]:
g = gens(G, 10)

B₄.₂ ∈ Sp{6,Int8}
 1  0  0  0  0  0
 0  1  0  0  0  0
 0  0  1  0  0  0
 0  1  0  1  0  0
 1  0  0  0  1  0
 0  0  0  0  0  1

In [32]:
PropertyT.grading(g)

Root in ℝ^3 of length √2
[-1, -1, 0]

In [33]:
Δs = PropertyT.laplacians(
    RG,
    S,
    x -> (gx = PropertyT.grading(x); Set([gx, -gx])),
);

According to **Definition 3.13** we define $\operatorname{Lev}_2^3$ as follows (note: `PropertyT.Adj(Δs, :A₁)` stands here for short squares).

In [34]:
Lev₂³ = PropertyT.Adj(Δs, :A₁) + PropertyT.Adj(Δs, :C₂)

696·(id) -32·A₁.₂ -32·A₁.₃ -32·A₂.₁ -32·A₂.₃ -32·A₃.₁ -32·A₃.₂ -48·B₄.₁ -48·B₅.₂ -48·B₆.₃ -32·B₄.₂ -32·B₄.₃ -32·B₅.₃ -48·B₁.₄ -48·B₂.₅ -48·B₃.₆ -32·B₂.₄ -32·B₃.₄ -32·B₃.₅ -32·A₁.₂^-1 -32·A₁.₃^-1 -32·A₂.₁^-1 -32·A₂.₃^-1 -32·A₃.₁^-1 -32·A₃.₂^-1 -48·B₄.₁^-1 -48·B₅.₂^-1 -48·B₆.₃^-1 -32·B₄.₂^-1 -32·B₄.₃^-1 -32·B₅.₃^-1 -48·B₁.₄^-1 -48·B₂.₅^-1 -48·B₃.₆^-1 -32·B₂.₄^-1 -32·B₃.₄^-1 -32·B₃.₅^-1 +1·A₁.₂^2 +1·A₁.₂*A₂.₁ +1·A₁.₂*B₄.₁ +2·A₁.₂*B₅.₂ +1·A₁.₂*B₄.₂ +2·A₁.₂*B₁.₄ +1·A₁.₂*B₂.₅ +1·A₁.₂*B₂.₄ +1·A₁.₂*A₂.₁^-1 +1·A₁.₂*B₄.₁^-1 +2·A₁.₂*B₅.₂^-1 +1·A₁.₂*B₄.₂^-1 +2·A₁.₂*B₁.₄^-1 +1·A₁.₂*B₂.₅^-1 +1·A₁.₂*B₂.₄^-1 +1·A₁.₃^2 +1·A₁.₃*A₃.₁ +1·A₁.₃*B₄.₁ +2·A₁.₃*B₆.₃ +1·A₁.₃*B₄.₃ +2·A₁.₃*B₁.₄ +1·A₁.₃*B₃.₆ +1·A₁.₃*B₃.₄ +1·A₁.₃*A₃.₁^-1 +1·A₁.₃*B₄.₁^-1 +2·A₁.₃*B₆.₃^-1 +1·A₁.₃*B₄.₃^-1 +2·A₁.₃*B₁.₄^-1 +1·A₁.₃*B₃.₆^-1 +1·A₁.₃*B₃.₄^-1 +1·A₂.₁*A₁.₂ +1·A₂.₁^2 +2·A₂.₁*B₄.₁ +1·A₂.₁*B₅.₂ +1·A₂.₁*B₄.₂ +1·A₂.₁*B₁.₄ +2·A₂.₁*B₂.₅ +1·A₂.₁*B₂.₄ +1·A₂.₁*A₁.₂^-1 +2·A₂.₁*B₄.₁^-1 +1·A₂.₁*B₅.₂^-1 +1·A₂.₁*B₄.₂^-1 +1·A₂.₁*B₁.₄^-1 +2·A₂.

In [35]:
Lev₃³ = PropertyT.Adj(Δs, :A₂) + PropertyT.Adj(Δs, Symbol("A₁×C₁"))

576·(id) -40·A₁.₂ -40·A₁.₃ -40·A₂.₁ -40·A₂.₃ -40·A₃.₁ -40·A₃.₂ -16·B₄.₁ -16·B₅.₂ -16·B₆.₃ -40·B₄.₂ -40·B₄.₃ -40·B₅.₃ -16·B₁.₄ -16·B₂.₅ -16·B₃.₆ -40·B₂.₄ -40·B₃.₄ -40·B₃.₅ -40·A₁.₂^-1 -40·A₁.₃^-1 -40·A₂.₁^-1 -40·A₂.₃^-1 -40·A₃.₁^-1 -40·A₃.₂^-1 -16·B₄.₁^-1 -16·B₅.₂^-1 -16·B₆.₃^-1 -40·B₄.₂^-1 -40·B₄.₃^-1 -40·B₅.₃^-1 -16·B₁.₄^-1 -16·B₂.₅^-1 -16·B₃.₆^-1 -40·B₂.₄^-1 -40·B₃.₄^-1 -40·B₃.₅^-1 +2·A₁.₂*A₁.₃ +1·A₁.₂*A₂.₃ +1·A₁.₂*A₃.₁ +2·A₁.₂*A₃.₂ +2·A₁.₂*B₆.₃ +1·A₁.₂*B₄.₃ +2·A₁.₂*B₅.₃ +2·A₁.₂*B₃.₆ +2·A₁.₂*B₃.₄ +1·A₁.₂*B₃.₅ +2·A₁.₂*A₁.₃^-1 +1·A₁.₂*A₂.₃^-1 +1·A₁.₂*A₃.₁^-1 +2·A₁.₂*A₃.₂^-1 +2·A₁.₂*B₆.₃^-1 +1·A₁.₂*B₄.₃^-1 +2·A₁.₂*B₅.₃^-1 +2·A₁.₂*B₃.₆^-1 +2·A₁.₂*B₃.₄^-1 +1·A₁.₂*B₃.₅^-1 +1·A₁.₃*A₂.₁ +2·A₁.₃*A₂.₃ +1·A₁.₃*A₃.₂ +2·A₁.₃*B₅.₂ +1·A₁.₃*B₄.₂ +2·A₁.₃*B₅.₃ +2·A₁.₃*B₂.₅ +2·A₁.₃*B₂.₄ +1·A₁.₃*B₃.₅ +2·A₁.₃*A₁.₂^-1 +1·A₁.₃*A₂.₁^-1 +2·A₁.₃*A₂.₃^-1 +1·A₁.₃*A₃.₂^-1 +2·A₁.₃*B₅.₂^-1 +1·A₁.₃*B₄.₂^-1 +2·A₁.₃*B₅.₃^-1 +2·A₁.₃*B₂.₅^-1 +2·A₁.₃*B₂.₄^-1 +1·A₁.₃*B₃.₅^-1 +1·A₂.₁*A₁.₃ +2·A₂.₁*A₂.₃ +2·A₂.₁*A₃.₁ +1·A₂.₁

## Optimization problem

### Symmetry reduction

In [36]:
import PropertyT.SA as StarAlgebras
import PropertyT.SW as SymbolicWedderburn
using PropertyT.PG # PermutationGroups

In [37]:
wd = let RG = RG, N = N
    G = StarAlgebras.object(RG)
    P = PermGroup(perm"(1,2)", Perm(circshift(1:N, -1)))
    Weyl = Groups.Constructions.WreathProduct(PermGroup(perm"(1,2)"), P)
    act = PropertyT.action_by_conjugation(G, Weyl)

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Weyl,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
    )
end
@info wd

  6.334770 seconds (22.13 M allocations: 1.386 GiB, 3.18% gc time, 201.94% compilation time)


┌ Info: Wedderburn Decomposition into 21968 orbits and 5 simple summands of sizes
└ [97, 56, 152, 94, 74]


In [38]:
@time model, varP = PropertyT.sos_problem_primal(
    Lev₂³ + Lev₃³,
    Δ,
    wd;
    augmented = true,
);
model

  4.377986 seconds (29.26 M allocations: 6.125 GiB, 13.44% gc time, 18.00% compilation time)


A JuMP Model
Maximization problem with:
Variables: 25218
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 21967 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ

In [41]:
include(joinpath(@__DIR__, "..", "src", "optimizers.jl"));
with_optimizer = scs_optimizer(;
    eps = 1e-10,
    max_iters = 20_000,
    accel = 50,
    alpha = 1.6,
);

In [42]:
warm = nothing

> **Note**: Running the following optimization step may take a few minutes.

In [43]:
status, warm = PropertyT.solve(
        model,
        with_optimizer,
        warm,
    );
@info "Optimization has finished with" status

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 25218, constraints m: 47184
cones: 	  z: primal zero / dual free vars: 21967
	  s: psd vars: 25217, ssize: 5
settings: eps_abs: 1.0e-10, eps_rel: 1.0e-10, eps_infeas: 1.0e-07
	  alpha: 1.60, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 20000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 50, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-mkl-pardiso
	  nnz(A): 301575, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 7.22e+01  9.99e-01  5.39e+02 -3.92e+02  1.00e-01  2.90e-01 
   250| 1.13e-02  6.63e-04  1.92e

┌ Info: Optimization has finished with
└   status = ITERATION_LIMIT::TerminationStatusCode = 11


### Reconstructing and certifying the solution
Since we symmetrized our problem, we don't have direct access to `P` or `Q` as defined above. So first we need to reconstruct `Q` from the action of `Weyl` on $\operatorname{Ball}(S, R)$:

In [44]:
@info "reconstructing the solution"
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

[ Info: reconstructing the solution


  0.336783 seconds (1.16 M allocations: 89.771 MiB, 7.58% gc time, 71.83% compilation time)


961×961 Matrix{Float64}:
 0.0   0.0           0.0           0.0          …   0.0           0.0
 0.0   4.23046       0.0844785    -0.111967        -0.00222208   -0.00666439
 0.0   0.0844785     4.23046       0.0231808       -0.00222208   -0.00666439
 0.0  -0.111967      0.0231808     4.23046          0.00194968   -0.00344572
 0.0   0.0231808     0.0844785     0.0844785        0.0086506     0.00451286
 0.0   0.0231808    -0.111967      0.0844785    …   0.00194968   -0.00344572
 0.0   0.0844785     0.0231808     0.0231808        0.0086506     0.00451286
 0.0   0.10433       0.10433      -0.0424452        0.01495      -0.00535707
 0.0  -0.0424452     0.0448794     0.10433         -0.0104408    -0.0162984
 0.0   0.0448794    -0.0424452     0.0448794       -0.0104408    -0.0162984
 0.0   0.00377729    0.0179707     0.00377729   …  -0.00200845   -0.00622942
 0.0   0.0179707     0.00377729    0.0858356       -0.00200845   -0.00622942
 0.0   0.0858356     0.0858356     0.0179707       -0.000517

In [45]:
@info "certifying the solution"
certified, λ = PropertyT.certify_solution(
    Lev₂³ + Lev₃³,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)

if certified && λ > 0
    @info "Certified result: Lev₂³ + Lev₃³ is positive" PropertyT.IntervalArithmetic.inf(λ)
else
    @info "Could NOT certify the positivity of Lev₂³ + Lev₃³" certified λ
end

[ Info: certifying the solution
┌ Info: Checking in Float64 arithmetic with
└   λ = 2.4901805502281795


  0.013496 seconds (6 allocations: 10.288 MiB)


┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ≈ -2.29543e-10
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ≈ 1.90382
└  λ ≈ -5.12509926406636
┌ Info: Could NOT certify the positivity of Lev₂³ + Lev₃³
│   certified = false
└   λ = -5.12509926406636


# $\texttt{C}_\texttt{2}$-graded $\operatorname{Adj}$

We wish to prove
> **Theorem 3.10** Let $G = \operatorname{Sp}_4(\mathbb{Z})$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{C}_\texttt{2}$ endowed with the set of Steinberg generators $S$. Let $V$ denote the ambient vector space of the root system. Then $$\operatorname{Adj}_V −\lambda \Delta_V ⩾_R 0$$
whenever $(\lambda, R) \in (0.24493, 3)$.

Proving this theorem for $R = 3$ requires quite a bit of memory (`~16GB`) and quite a bit of patience.

In [46]:
N = 2
G = MatrixGroups.SymplecticGroup{2*N}(Int8)

group of 4×4 symplectic matrices

In [47]:
HALFRADIUS = 3
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

[ Info: generating wl-metric ball of radius 6


  3.351726 seconds (49.50 M allocations: 1.682 GiB, 15.35% gc time)


[ Info: sizes = [17, 209, 2073, 18313, 149449, 1148696]
[ Info: computing the *-algebra structure for G


  0.454616 seconds (6.80 M allocations: 250.062 MiB)
  3.806987 seconds (56.30 M allocations: 1.927 GiB, 13.51% gc time)


In [48]:
S

16-element Vector{FPGroupElement{Groups.MatrixGroups.SymplecticGroup{4, Int8, DataType, Groups.MatrixGroups.ElementarySymplectic{4, Int8}}, KnuthBendix.Words.Word{UInt8}}}:
 A₁.₂
 A₂.₁
 B₃.₁
 B₄.₂
 B₃.₂
 B₁.₃
 B₂.₄
 B₂.₃
 A₁.₂^-1
 A₂.₁^-1
 B₃.₁^-1
 B₄.₂^-1
 B₃.₂^-1
 B₁.₃^-1
 B₂.₄^-1
 B₂.₃^-1

In [49]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

16·(id) -1·A₁.₂ -1·A₂.₁ -1·B₃.₁ -1·B₄.₂ -1·B₃.₂ -1·B₁.₃ -1·B₂.₄ -1·B₂.₃ -1·A₁.₂^-1 -1·A₂.₁^-1 -1·B₃.₁^-1 -1·B₄.₂^-1 -1·B₃.₂^-1 -1·B₁.₃^-1 -1·B₂.₄^-1 -1·B₂.₃^-1

## Grading

Let us define $\texttt{C}_\texttt{2}$ grading for $(G, S)$. That is, to every generator $s \in S$ we need to assign a root in $\texttt{C}_\texttt{2}$ in a compatible fashion. For the precise description of this grading see **Example 3.2 (2)**.  

In [50]:
using PropertyT.Roots

In [51]:
g = gens(G, 1)

A₁.₂ ∈ Sp{4,Int8}
 1  1   0  0
 0  1   0  0
 0  0   1  0
 0  0  -1  1

In [52]:
PropertyT.grading(g)

Root in ℝ^2 of length √2
[1, -1]

In [53]:
g = gens(G, 3)

B₃.₁ ∈ Sp{4,Int8}
 1  0  0  0
 0  1  0  0
 1  0  1  0
 0  0  0  1

In [54]:
PropertyT.grading(g)

Root in ℝ^2 of length 2
[-2, 0]

In [55]:
g = gens(G, 5)

B₃.₂ ∈ Sp{4,Int8}
 1  0  0  0
 0  1  0  0
 0  1  1  0
 1  0  0  1

In [56]:
PropertyT.grading(g)

Root in ℝ^2 of length √2
[-1, -1]

In [57]:
Δs = PropertyT.laplacians(
    RG,
    S,
    x -> (gx = PropertyT.grading(x); Set([gx, -gx])),
);

Here `Δs` is just a map from lines in the root system $\Omega = \texttt{C}_{\texttt{2}}$ to the corresponding Laplacians, e.g. below we can see that to the line through `α = [1, -1]` and `-α` (and the origin) we assign
$$ \Delta_{Lα} = 4 - A_{1,2} - A_{2,1} - A_{1,2}^{-1} - A_{2,1}^{-1}.$$ 

In [58]:
let α = Root([1,-1])
    Lα = Set([α, -α])
    Δs[Lα]
end

4·(id) -1·A₁.₂ -1·A₂.₁ -1·A₁.₂^-1 -1·A₂.₁^-1

In [59]:
let α = Root([2,0])
    Lα = Set([α, -α])
    Δs[Lα]
end

4·(id) -1·B₃.₁ -1·B₁.₃ -1·B₃.₁^-1 -1·B₁.₃^-1

In [60]:
let α = Root([-1,-1])
    Lα = Set([α, -α])
    Δs[Lα]
end

4·(id) -1·B₃.₂ -1·B₂.₃ -1·B₃.₂^-1 -1·B₂.₃^-1

Following the definition of $\operatorname{Adj}$ we define
$$ \operatorname{Adj}_{\texttt{C}_\texttt{2}} = 
\prod_{
    \langle L\alpha, L\beta \rangle \cap \Omega \cong \texttt{C}_{\texttt{2}}
} \Delta_{L\alpha} \Delta_{L\beta} $$

In [61]:
AdjC₂ = PropertyT.Adj(Δs, :C₂)

192·(id) -24·A₁.₂ -24·A₂.₁ -24·B₃.₁ -24·B₄.₂ -24·B₃.₂ -24·B₁.₃ -24·B₂.₄ -24·B₂.₃ -24·A₁.₂^-1 -24·A₂.₁^-1 -24·B₃.₁^-1 -24·B₄.₂^-1 -24·B₃.₂^-1 -24·B₁.₃^-1 -24·B₂.₄^-1 -24·B₂.₃^-1 +1·A₁.₂*B₃.₁ +2·A₁.₂*B₄.₂ +1·A₁.₂*B₃.₂ +2·A₁.₂*B₁.₃ +1·A₁.₂*B₂.₄ +1·A₁.₂*B₂.₃ +1·A₁.₂*B₃.₁^-1 +2·A₁.₂*B₄.₂^-1 +1·A₁.₂*B₃.₂^-1 +2·A₁.₂*B₁.₃^-1 +1·A₁.₂*B₂.₄^-1 +1·A₁.₂*B₂.₃^-1 +2·A₂.₁*B₃.₁ +1·A₂.₁*B₄.₂ +1·A₂.₁*B₃.₂ +1·A₂.₁*B₁.₃ +2·A₂.₁*B₂.₄ +1·A₂.₁*B₂.₃ +2·A₂.₁*B₃.₁^-1 +1·A₂.₁*B₄.₂^-1 +1·A₂.₁*B₃.₂^-1 +1·A₂.₁*B₁.₃^-1 +2·A₂.₁*B₂.₄^-1 +1·A₂.₁*B₂.₃^-1 +1·B₃.₁*A₁.₂ +2·B₃.₁*B₄.₂ +2·B₃.₁*B₃.₂ +2·B₃.₁*B₂.₄ +1·B₃.₁*B₂.₃ +1·B₃.₁*A₁.₂^-1 +2·B₃.₁*A₂.₁^-1 +2·B₃.₁*B₄.₂^-1 +2·B₃.₁*B₃.₂^-1 +2·B₃.₁*B₂.₄^-1 +1·B₃.₁*B₂.₃^-1 +1·B₄.₂*A₂.₁ +2·B₄.₂*B₃.₂ +2·B₄.₂*B₁.₃ +1·B₄.₂*B₂.₃ +2·B₄.₂*A₁.₂^-1 +1·B₄.₂*A₂.₁^-1 +2·B₄.₂*B₃.₁^-1 +2·B₄.₂*B₃.₂^-1 +2·B₄.₂*B₁.₃^-1 +1·B₄.₂*B₂.₃^-1 +1·B₃.₂*A₁.₂ +1·B₃.₂*A₂.₁ +1·B₃.₂*B₁.₃ +1·B₃.₂*B₂.₄ +1·B₃.₂*A₁.₂^-1 +1·B₃.₂*A₂.₁^-1 +2·B₃.₂*B₃.₁^-1 +2·B₃.₂*B₄.₂^-1 +1·B₃.₂*B₁.₃^-1 +1·B₃.₂*B₂.₄^-1 +1·B₁.₃*A₂.₁ +1·B₁

It is not hard to see that for $\Omega = \texttt{C}_{\texttt{2}}$ 
 * we are simply looking at products of all $\Delta_{L\alpha}$ and $\Delta_{L\beta}$ where $L\alpha \neq L\beta$, and
 * that the new definition is an analouge to the definition of $\operatorname{Adj}$ from [On property (T) for $\operatorname{Aut}(F_n)$ and $\operatorname{SL}_n(\mathbb{Z})$](https://arxiv.org/abs/1812.03456).

In [62]:
AdjC₂ == Δ^2 - sum(Δs[Lα]^2 for Lα in keys(Δs))

true

## Optimization problem

### Symmetry reduction

In [63]:
import PropertyT.SA as StarAlgebras
import PropertyT.SW as SymbolicWedderburn
using PropertyT.PG # PermutationGroups

In [64]:
wd = let RG = RG, N = N
    G = StarAlgebras.object(RG)
    P = PermGroup(perm"(1,2)", Perm(circshift(1:N, -1)))
    Weyl = Groups.Constructions.WreathProduct(PermGroup(perm"(1,2)"), P)
    act = PropertyT.action_by_conjugation(G, Weyl)

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Weyl,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
    )
end
@info wd

  0.440349 seconds (31.14 M allocations: 1.409 GiB, 12.69% compilation time: 56% of which was recompilation)


┌ Info: Wedderburn Decomposition into 289560 orbits and 4 simple summands of sizes
└ [590, 549, 484, 450]


In [65]:
@time model, varP = PropertyT.sos_problem_primal(
    AdjC₂,
    Δ,
    wd;
    augmented = true,
);
model

 39.593111 seconds (217.89 M allocations: 123.553 GiB, 12.32% gc time)


A JuMP Model
Maximization problem with:
Variables: 544166
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 289559 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ

## Solving the problem numerically
We will use `scs` [Splitting Conic Solver](https://github.com/cvxgrp/scs) so solve this problem.

In [66]:
include(joinpath(@__DIR__, "..", "src", "optimizers.jl"));
with_optimizer = scs_optimizer(;
    eps = 1e-9,
    max_iters = 20_000,
    accel = 50,
    alpha = 1.95,
);

In [67]:
warm = nothing

> **Note** 
> * To obtain just any positive lower bound it is advisable to (artificially) bound the objective from above, e.g. by adding
    ```julia
    JuMP.@constraint(model, upper_bound, model[:λ] ≤ 0.125)
    ```
    before solving the model (to bring the solve time to below 1h).
> * If you do not bound the objective you will need to re-run the cell below several times to obtain `status = OPTIMAL::TerminationStatusCode = 1`. To succesfully certify **a lower bound** that might not be necessary. However this will be necessary to obtain **the bound advertised** in the paper.


In [68]:
JuMP.@constraint(model, upper_bound, model[:λ] ≤ 0.125)

upper_bound : λ ≤ 0.125

In [69]:
status, warm = PropertyT.solve(
        model,
        with_optimizer,
        warm,
    );
# note: since we're using scs there will be no printout until the optimization has finished 
# please bear with us...
@info "Optimization has finished with" status

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 544166, constraints m: 833725
cones: 	  z: primal zero / dual free vars: 289559
	  l: linear vars: 1
	  s: psd vars: 544165, ssize: 4
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.95, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 20000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 50, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-mkl-pardiso
	  nnz(A): 4965179, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.40e+01  1.00e+00  2.03e+01 -1.02e+01  1.00e-01  1.06e+02 
   250|

┌ Info: Optimization has finished with
└   status = OPTIMAL::TerminationStatusCode = 1


### Reconstructing and certifying the solution

In [70]:
@info "reconstructing the solution"
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

[ Info: reconstructing the solution


  1.584577 seconds (22.84 M allocations: 861.932 MiB, 11.93% gc time, 4.39% compilation time)


2073×2073 Matrix{Float64}:
 0.0   0.0           0.0           0.0          …   0.0           0.0
 0.0   0.53076       0.122576      0.127844         0.00062125   -6.15139e-5
 0.0   0.122576      0.53076       0.13139          0.00062125   -6.15139e-5
 0.0   0.127844      0.13139       0.527937         0.000852002  -0.000178643
 0.0   0.13139       0.127844      0.129358         0.000852002  -0.000178643
 0.0   0.127784      0.127784      0.131408     …  -2.42835e-5   -0.000543139
 0.0   0.13139       0.127844      0.12953          0.0010291     0.00106489
 0.0   0.127844      0.13139       0.132646         0.0010291     0.00106489
 0.0   0.127784      0.127784      0.127877         0.000569626   0.00228599
 0.0   0.119758      0.122304      0.127844         0.00062125   -6.15139e-5
 0.0   0.122304      0.119758      0.13139      …   0.00062125   -6.15139e-5
 0.0   0.127844      0.13139       0.121956         0.000852002  -0.000178643
 0.0   0.13139       0.127844      0.132655         

In [71]:
@info "certifying the solution"
certified, λ = PropertyT.certify_solution(
    AdjC₂,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)

if certified && λ > 0
    @info "Certified result: Adj_C₂ is positive" PropertyT.IntervalArithmetic.inf(λ)
else
    @info "Could NOT certify the positivity of Adj_C₂" certified λ
end

[ Info: certifying the solution
┌ Info: Checking in Float64 arithmetic with
└   λ = 0.1249999976381011


  0.302645 seconds (6 allocations: 41.550 MiB)


┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ≈ 4.70877e-11
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ≈ 5.04044e-6
└  λ ≈ 0.12491935055025002
┌ Info: Checking in Interval{Float64} arithmetic with
└   λ_int = [0.124999, 0.125]


  2.342842 seconds (67 allocations: 804.395 MiB, 6.83% gc time)


┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ∈ [-6.00359e-08, 6.0127e-08]
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ∈ [4.98036e-06, 5.10053e-06]
└  λ ∈ [0.124918, 0.124921]
┌ Info: Certified result: Adj_C₂ is positive
└   PropertyT.IntervalArithmetic.inf(λ) = 0.12491838921433493


In [72]:
using Dates
Dates.now()

2024-07-08T15:43:41.262

In [73]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × AMD Ryzen 7 7840U w/ Radeon  780M Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, znver3)
Threads: 8 default, 0 interactive, 4 GC (on 16 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
